That notebook is going to be used for the capstone project of IBM data science certification course

In [96]:
import pandas as pd
import numpy as np

In [97]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


<h2>Setup </h2>
<p>BeautifulSoup library is used for web scraping. It is imported as soup<p>
To access the wikipedia web page, urlopen is imported from urllib 

In [98]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen

In [99]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [100]:
wiki_data = urlopen(wiki_url)
wiki_html = wiki_data.read()
wiki_data.close()

<h2>Web Scraping</h2><p>Following cells perform the web scraping. From Wikipedia file, the table element scraped by its class name<p>

In [101]:
page_soup=soup(wiki_html, 'html.parser')

In [102]:
postCode_table=page_soup.findAll('table', {'class':"wikitable sortable"})

In [105]:
postCode_table = postCode_table[0]

Name of the columns are extracted from html table

In [170]:
cols = postCode_table.findAll('th', {})
print(cols)

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


<p>Tags of items in table are removed by using following for loop<p>

In [107]:
cols_name = []
for col in cols:
    cols_name.append(col.text)
print(cols_name)

['Postcode', 'Borough', 'Neighbourhood\n']


In [108]:
cols_name[2] = cols_name[2].replace('\n','')

In [109]:
type(cols_name)

list

Items under each columns are extracted from the html table

In [110]:
all_rows = postCode_table.findAll('tr', {})

In [111]:
all_rows=all_rows[1:]

In [112]:
first_row = all_rows[0]

In [113]:
first_row_data = first_row.findAll('td',{})

In [114]:
first_row_data

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>]

In [115]:
rows_data = []
for data in first_row_data:
    rows_data.append(data.text)
print(rows_data)

['M1A', 'Not assigned', 'Not assigned\n']


In [116]:
rows_data[2] = rows_data[2].replace('\n','')

In [117]:
type(rows_data)

list

Function to handle each and every single rows of columns from the table

In [118]:
def row_gen(row):
    row_data = row.findAll('td',{})
    r_data = []
    for d in row_data:
        r_data.append(d.text)
    r_data[2] = r_data[2].replace('\n','')
    return r_data

Gathering every rows generated

In [122]:
rows_tobe = []
for i in all_rows:
    rows_tobe.append(row_gen(i))

A datafrane created by all of rows and columns 

In [123]:
df_rows = pd.DataFrame(rows_tobe,columns = cols_name)

In [124]:
df_rows.head(5) #to Check the general appearence of table from head 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [125]:
df_rows.tail(5)  #to Check the general appearence of table from head 

,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


Not assigned boroughs are removed from dataframe

In [126]:
df_rows = df_rows[df_rows['Borough'] != 'Not assigned']

Not assigned Neighbourhoods are assigned as their borough's name

In [127]:
for i in df_rows.index:
    if df_rows.loc[i,'Neighbourhood'] == 'Not assigned':
        df_rows.loc[i,'Neighbourhood'] = df_rows.loc[i,'Borough']
        

In [128]:
df_rows[df_rows['Neighbourhood']==df_rows['Borough']]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


Items are grouped by their postcodes and then neighbourhoods with same postal code are combined into a single row

In [142]:
df_rows= df_rows.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [160]:
df_rows.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>Accessing the location data</h2>
<p>Because of unreliable package, continuing with csv file is decided after a couple of tries. <p>
<p>Wget which we commonly used in labs installed, in order to download csv provided.<p>

In [50]:
conda install -c anaconda wget

Solving environment: done

## Package Plan ##

  environment location: /Users/oguzonuk/opt/anaconda3

  added / updated specs:
    - wget


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.8.28  |                0         133 KB  anaconda
    certifi-2019.9.11          |           py37_0         154 KB  anaconda
    conda-4.7.12               |           py37_0         3.0 MB  anaconda
    openssl-1.1.1d             |       h1de35cc_2         3.4 MB  anaconda
    wget-1.20.1                |       h051b688_0         478 KB  anaconda
    ------------------------------------------------------------
                                           Total:         7.2 MB

The following NEW packages will be INSTALLED:

  wget               anaconda/osx-64::wget-1.20.1-h051b688_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates           

In [51]:
!wget -O coordinates_neighbourhood.csv http://cocl.us/Geospatial_data

--2019-11-14 23:45:57--  http://cocl.us/Geospatial_data
Resolving cocl.us... 159.8.72.228
Connecting to cocl.us|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-11-14 23:45:57--  https://cocl.us/Geospatial_data
Connecting to cocl.us|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-14 23:45:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 185.235.236.197
Connecting to ibm.box.com|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-14 23:46:00--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:44

A dataframe which consists location data of each item is created

In [53]:
df_coord = pd.read_csv("coordinates_neighbourhood.csv")
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [161]:
df_coord.shape

(103, 3)

Coordinates are concatenated to main dataframe to obtain requested dataframe 

In [162]:
df_rows = pd.concat([df_rows,df_coord['Latitude']],axis = 1)

In [163]:
df_rows = pd.concat([df_rows,df_coord['Longitude']],axis = 1)

In [164]:
df_rows.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h2>Shape of the main dataframe is as follows </h2>

In [167]:
df_rows.shape

(103, 5)